In [7]:
import sympy as sp
import numpy as np

from sumpy.expansion.diff_op import (
    laplacian,
    make_identity_diff_op,
)

from sumpy.recurrence import _make_sympy_vec


In [8]:
#Create Hankel Function

from sympy import hankel1
z = sp.symbols("z")
f = hankel1(0, z)

In [9]:
f.diff(z)

hankel1(-1, z)/2 - hankel1(1, z)/2

In [10]:
var = _make_sympy_vec("x", 2)
var_t = _make_sympy_vec("t", 2)
k = 1
abs_dist = sp.sqrt((var[0]-var_t[0])**2 + (var[1]-var_t[1])**2)
g_x_y = (1j/4) * hankel1(0, k * abs_dist)


In [12]:
g_x_y

0.25*I*hankel1(0, sqrt((-t0 + x0)**2 + (-t1 + x1)**2))

In [13]:
derivs = [sp.diff(g_x_y, var_t[0], i).subs(var_t[0], 0).subs(var_t[1], 0) for i in range(6)]

In [14]:
derivs

[0.25*I*hankel1(0, sqrt(x0**2 + x1**2)),
 -0.25*I*x0*(hankel1(-1, sqrt(x0**2 + x1**2))/2 - hankel1(1, sqrt(x0**2 + x1**2))/2)/sqrt(x0**2 + x1**2),
 0.0625*I*(x0**2*(hankel1(-2, sqrt(x0**2 + x1**2)) - 2*hankel1(0, sqrt(x0**2 + x1**2)) + hankel1(2, sqrt(x0**2 + x1**2)))/(x0**2 + x1**2) - 2*x0**2*(hankel1(-1, sqrt(x0**2 + x1**2)) - hankel1(1, sqrt(x0**2 + x1**2)))/(x0**2 + x1**2)**(3/2) + 2*(hankel1(-1, sqrt(x0**2 + x1**2)) - hankel1(1, sqrt(x0**2 + x1**2)))/sqrt(x0**2 + x1**2)),
 0.03125*I*x0*(4*x0**2*(hankel1(-2, sqrt(x0**2 + x1**2)) - 2*hankel1(0, sqrt(x0**2 + x1**2)) + hankel1(2, sqrt(x0**2 + x1**2)))/(x0**2 + x1**2)**2 - 12*x0**2*(hankel1(-1, sqrt(x0**2 + x1**2)) - hankel1(1, sqrt(x0**2 + x1**2)))/(x0**2 + x1**2)**(5/2) - 4*(hankel1(-2, sqrt(x0**2 + x1**2)) - 2*hankel1(0, sqrt(x0**2 + x1**2)) + hankel1(2, sqrt(x0**2 + x1**2)))/(x0**2 + x1**2) - (x0**2*(hankel1(-3, sqrt(x0**2 + x1**2)) - 2*hankel1(-1, sqrt(x0**2 + x1**2)) + hankel1(1, sqrt(x0**2 + x1**2)))/(x0**2 + x1**2) - x0**2*(han